# 🏃 03. Running & Managing Containers

## 📑 Meta-Intro
- **Key Topics:** Container Lifecycle (`run`, `stop`, `rm`), Detached Mode (`-d`), Port Mapping (`-p`), and Interactive Shells (`exec -it`).
- **Executive Summary:** Now that Docker is installed, we move to the core daily tasks of a DevOps engineer: spinning up services. A container is ephemeral—it is designed to be created, stopped, and destroyed rapidly. In this notebook, we will master the `docker run` command, learn how to expose container services to your local machine using ports, and how to "enter" a running container to debug it.

---

## 1. 🔄 The Container Lifecycle

Unlike a VM that you boot once and keep running for years, containers are often short-lived. They follow a strict state machine.

1.  **Create:** The container is built from an image but not running.
2.  **Start:** The process inside (PID 1) begins execution.
3.  **Running:** The process is active.
4.  **Paused/Stopped:** The process is frozen or killed (SIGTERM).
5.  **Destroyed:** The container filesystem is removed.

### The Magic Command: `docker run`
`docker run` is actually a combination of `create` + `start`.

```bash
# Basic Syntax
docker run <image_name>
```

---

## 2. 👻 Modes: Interactive vs. Detached

By default, Docker runs in the foreground (attached). If you close your terminal, the container dies.

### Detached Mode (`-d`)
Runs the container in the background. It prints the Container ID and returns control to your terminal immediately. This is how servers (Nginx, Database) are run.

```bash
# Run Nginx in background
docker run -d --name my-web-server nginx
```

### Interactive Mode (`-it`)
Used when you need to interact with the container directly (like a Python shell or Ubuntu terminal).
- `-i` (Interactive): Keep STDIN open.
- `-t` (TTY): Allocate a pseudo-terminal.

---

## 3. 🌐 Networking: Port Mapping (`-p`)

By default, a container is isolated. Even if Nginx is running on port 80 *inside* the container, you cannot access it from your laptop.

We must **publish** the port using `-p HOST_PORT:CONTAINER_PORT`.



### Example:
```bash
# Map Host Port 8080 to Container Port 80
docker run -d -p 8080:80 nginx
```
Now, if you go to `http://localhost:8080` in your browser, the request travels:
`Your Browser` -> `Host :8080` -> `Docker Bridge` -> `Container :80` -> `Nginx`.

---

## 4. 🕵️‍♀️ Debugging: Logs and Exec

When a container runs in detached mode (`-d`), you can't see its output. We use management commands to inspect it.

| Command | Purpose |
| :--- | :--- |
| `docker logs <id>` | View the stdout/stderr output (useful for crash logs). |
| `docker exec -it <id> bash` | "SSH" into the running container to explore files. |
| `docker inspect <id>` | View low-level JSON metadata (IP address, volumes, env vars). |

---

## 🧹 5. Cleanup: Preventing Disk Bloat

Every time you run `docker run`, a new container is created. If you don't delete them, they eat up disk space.

* **`docker stop <id>`**: Stops the process (files remain).
* **`docker rm <id>`**: Removes the container filesystem.
* **`docker run --rm`**: Automatically removes the container when it exits (Great for scripts).
* **`docker system prune`**: The nuclear option. Deletes all stopped containers, unused networks, and dangling images.

---

## 🌊 Mini-Challenge: The Web Server

**Goal:** Spin up a web server and modify its content live.

**Instructions:**
1.  Run an Nginx container in detached mode, mapping host port **3000** to container port **80**.
2.  Verify it works by opening `localhost:3000`.
3.  Use `docker exec` to enter the container.
4.  Inside the container, run `echo "Hello CSE" > /usr/share/nginx/html/index.html`.
5.  Refresh your browser. Did the page change?
6.  **Cleanup:** Stop and remove the container.

---

In [ ]:
%%bash
# SOLUTION REFERENCE

# 1. Run Nginx detached
docker run -d --name cse-web -p 3000:80 nginx

# 2. Check if it's running
docker ps

# 3. Modify content (One-liner exec)
docker exec cse-web sh -c "echo 'Hello CSE Intake 24' > /usr/share/nginx/html/index.html"

# 4. Verify with curl (or browser)
curl localhost:3000

# 5. Cleanup
docker rm -f cse-web

## 🎓 Core Insight for Your CSE Career

**Ephemeral Nature**

A common mistake students make is treating a container like a "Small VM"—installing updates, editing config files, and saving data inside it (like we just did in the challenge). **This is an anti-pattern.** 

If you delete the container `cse-web`, your "Hello CSE" file is gone forever. In the next notebook, we will learn about **Volumes**, which is the *only* correct way to store data that needs to survive the destruction of a container.